# ಸೆಮ್ಯಾಂಟಿಕ್ ಕರ್ಣಲ್ ಮತ್ತು ಓಪನ್BnB MCP ಸರ್ವರ್ ಇಂಟಿಗ್ರೇಶನ್

ಈ ನೋಟುಬುಕ್ ಸೆಮ್ಯಾಂಟಿಕ್ ಕರ್ಣಲ್ ಅನ್ನು ನಿಜವಾದ ಓಪನ್BnB MCP ಸರ್ವರ್ ಜೊತೆಗೆ MCPStdioPlugin ಬಳಸಿ ನಿಜವಾದ Airbnb ವಾಸಸ್ಥಾನಗಳನ್ನು ಹುಡುಕಲು ಹೇಗೆ ಬಳಸಬಹುದು ಎಂಬುದನ್ನು ತೋರಿಸುತ್ತದೆ. LLM ಪ್ರವೇಶಕ್ಕಾಗಿ, ಇದು Azure AI Foundry ಅನ್ನು ಬಳಸುತ್ತದೆ. ನಿಮ್ಮ ಪರಿಸರ ಚರಾಂಶಗಳನ್ನು ಸೆಟ್ ಅಪ್ ಮಾಡಲು, ನೀವು [ಸೆಟ್ ಅಪ್ ಪಾಠ](/00-course-setup/README.md) ಅನ್ನು ಅನುಸರಿಸಬಹುದು.


## ಅಗತ್ಯವಿರುವ ಪ್ಯಾಕೇಜ್‌ಗಳನ್ನು ಆಮದುಮಾಡಿ


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP ಪ್ಲಗಿನ್ ಸಂಪರ್ಕವನ್ನು ರಚಿಸುವುದು

ನಾವು [OpenBnB MCP ಸರ್ವರ್](https://github.com/openbnb-org/mcp-server-airbnb) ಗೆ MCPStdioPlugin ಬಳಸಿ ಸಂಪರ್ಕಿಸುತ್ತೇವೆ. ಈ ಸರ್ವರ್ @openbnb/mcp-server-airbnb ಪ್ಯಾಕೇಜ್ ಮೂಲಕ Airbnb ಶೋಧ ಕಾರ್ಯಕ್ಷಮತೆಯನ್ನು ಒದಗಿಸುತ್ತದೆ.


## ಕ್ಲೈಂಟ್ ಅನ್ನು ರಚಿಸುವುದು

ಈ ಮಾದರಿಯಲ್ಲಿ, ನಾವು ನಮ್ಮ LLM ಪ್ರವೇಶಕ್ಕಾಗಿ Azure AI Foundry ಅನ್ನು ಬಳಸುತ್ತೇವೆ. ನಿಮ್ಮ ಪರಿಸರ ಚರಾಂಶಗಳು ಸರಿಯಾಗಿ ಹೊಂದಿಸಲಾಗಿದೆ ಎಂದು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಿ.


## ಪರಿಸರ ಸಂರಚನೆ

Azure OpenAI ಸೆಟ್ಟಿಂಗ್‌ಗಳನ್ನು ಸಂರಚಿಸಿ. ನೀವು ಕೆಳಗಿನ ಪರಿಸರ ಚರಾಂಶಗಳನ್ನು ಹೊಂದಿರುವುದನ್ನು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಿ:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ಇಂಟಿಗ್ರೇಶನ್ ಅನ್ನು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವುದು

ಈ ನೋಟು **ನಿಜವಾದ OpenBnB MCP ಸರ್ವರ್** ಗೆ ಸಂಪರ್ಕಿಸುತ್ತದೆ, ಇದು ನಿಜವಾದ Airbnb ಹುಡುಕಾಟ ಕಾರ್ಯಕ್ಷಮತೆಯನ್ನು ಒದಗಿಸುತ್ತದೆ.

### ಇದು ಹೇಗೆ ಕೆಲಸ ಮಾಡುತ್ತದೆ:

1. **MCPStdioPlugin**: MCP ಸರ್ವರ್‌ನೊಂದಿಗೆ ಸ್ಟ್ಯಾಂಡರ್ಡ್ ಇನ್‌ಪುಟ್/ಔಟ್‌ಪುಟ್ ಸಂವಹನವನ್ನು ಬಳಸುತ್ತದೆ
2. **ನಿಜವಾದ NPM ಪ್ಯಾಕೇಜ್**: `@openbnb/mcp-server-airbnb` ಅನ್ನು npx ಮೂಲಕ ಡೌನ್‌ಲೋಡ್ ಮಾಡಿ ಮತ್ತು ಚಲಾಯಿಸುತ್ತದೆ
3. **ಲೈವ್ ಡೇಟಾ**: ಅವರ API ಗಳಿಂದ ನಿಜವಾದ Airbnb ಆಸ್ತಿ ಡೇಟಾವನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತದೆ
4. **ಫಂಕ್ಷನ್ ಡಿಸ್ಕವರಿ**: ಏಜೆಂಟ್ MCP ಸರ್ವರ್‌ನಿಂದ ಲಭ್ಯವಿರುವ ಕಾರ್ಯಗಳನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಪತ್ತೆಹಚ್ಚುತ್ತದೆ

### ಲಭ್ಯವಿರುವ ಕಾರ್ಯಗಳು:

OpenBnB MCP ಸರ್ವರ್ ಸಾಮಾನ್ಯವಾಗಿ ಈ ಕೆಳಗಿನ ಕಾರ್ಯಗಳನ್ನು ಒದಗಿಸುತ್ತದೆ:
- **search_listings** - ಸ್ಥಳ ಮತ್ತು ಮಾನದಂಡಗಳ ಆಧಾರದ ಮೇಲೆ Airbnb ಆಸ್ತಿಗಳನ್ನು ಹುಡುಕಿ
- **get_listing_details** - ನಿರ್ದಿಷ್ಟ ಆಸ್ತಿಗಳ ಬಗ್ಗೆ ವಿವರವಾದ ಮಾಹಿತಿಯನ್ನು ಪಡೆಯಿರಿ
- **check_availability** - ನಿರ್ದಿಷ್ಟ ದಿನಾಂಕಗಳಿಗೆ ಲಭ್ಯತೆಯನ್ನು ಪರಿಶೀಲಿಸಿ
- **get_reviews** - ಆಸ್ತಿಗಳಿಗಾಗಿ ವಿಮರ್ಶೆಗಳನ್ನು ಪಡೆಯಿರಿ
- **get_host_info** - ಆಸ್ತಿ ಹೋಸ್ಟ್‌ಗಳ ಬಗ್ಗೆ ಮಾಹಿತಿಯನ್ನು ಪಡೆಯಿರಿ

### ಪೂರ್ವಶರತ್ತುಗಳು:

- ನಿಮ್ಮ ಸಿಸ್ಟಮ್‌ನಲ್ಲಿ **Node.js** ಇನ್‌ಸ್ಟಾಲ್ ಮಾಡಿರಬೇಕು
- MCP ಸರ್ವರ್ ಪ್ಯಾಕೇಜ್ ಡೌನ್‌ಲೋಡ್ ಮಾಡಲು **ಇಂಟರ್ನೆಟ್ ಸಂಪರ್ಕ** ಅಗತ್ಯವಿದೆ
- **NPX** ಲಭ್ಯವಿರಬೇಕು (ಇದು Node.js ಜೊತೆಗೆ ಬರುತ್ತದೆ)

### ಸಂಪರ್ಕವನ್ನು ಪರೀಕ್ಷಿಸುವುದು:

ನೀವು MCP ಸರ್ವರ್ ಅನ್ನು ಕೆಳಗಿನಂತೆ ಕೈಯಾರೆ ಪರೀಕ್ಷಿಸಬಹುದು:
```bash
npx -y @openbnb/mcp-server-airbnb
```

ಇದು OpenBnB MCP ಸರ್ವರ್ ಅನ್ನು ಡೌನ್‌ಲೋಡ್ ಮಾಡಿ ಪ್ರಾರಂಭಿಸುತ್ತದೆ, ಇದಕ್ಕೆ ನಂತರ ಸೆಮ್ಯಾಂಟಿಕ್ ಕರ್ಣಲ್ ನಿಜವಾದ Airbnb ಡೇಟಾವಿಗಾಗಿ ಸಂಪರ್ಕಿಸುತ್ತದೆ.


## OpenBnB MCP ಸರ್ವರ್‌ನೊಂದಿಗೆ ಏಜೆಂಟ್ ಅನ್ನು ಚಲಾಯಿಸುವುದು

ಈಗ ನಾವು OpenBnB MCP ಸರ್ವರ್‌ಗೆ ಸಂಪರ್ಕ ಹೊಂದುವ AI ಏಜೆಂಟ್ ಅನ್ನು ಚಲಾಯಿಸುತ್ತೇವೆ, ಇದು 2 ವಯಸ್ಕರು ಮತ್ತು 1 ಮಗುಗಳಿಗೆ ಸ್ಟಾಕ್‌ಹೋಮ್‌ನಲ್ಲಿ ನಿಜವಾದ Airbnb ವಸತಿಗಳನ್ನು ಹುಡುಕುತ್ತದೆ. ಹುಡುಕಾಟದ ಮಾನದಂಡವನ್ನು ಬದಲಾಯಿಸಲು `user_inputs` ಪಟ್ಟಿ ಬದಲಾಯಿಸಲು ಮುಕ್ತವಾಗಿರಿ.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# ಸಾರಾಂಶ
ಅಭಿನಂದನೆಗಳು! ನೀವು Model Context Protocol (MCP) ಬಳಸಿ ನೈಜ ಜಗತ್ತಿನ ವಸತಿ ಹುಡುಕಾಟವನ್ನು ಒಗ್ಗೂಡಿಸುವ AI ಏಜೆಂಟ್ ಅನ್ನು ಯಶಸ್ವಿಯಾಗಿ ನಿರ್ಮಿಸಿದ್ದೀರಿ:

## ಬಳಸಿದ ತಂತ್ರಜ್ಞಾನಗಳು:
- ಸೆಮಾಂಟಿಕ್ ಕರ್ಣಲ್ - Azure OpenAI ಸಹಾಯದಿಂದ ಬುದ್ಧಿವಂತ ಏಜೆಂಟ್‌ಗಳನ್ನು ನಿರ್ಮಿಸಲು
- Azure AI Foundry - LLM ಸಾಮರ್ಥ್ಯಗಳು ಮತ್ತು ಚಾಟ್ ಪೂರ್ಣಗೊಳಿಸಲು
- MCP (Model Context Protocol) - ಮಾನಕೀಕೃತ ಸಾಧನ ಒಗ್ಗೂಡಿಸಲು
- OpenBnB MCP ಸರ್ವರ್ - ನೈಜ Airbnb ಹುಡುಕಾಟ ಕಾರ್ಯಕ್ಷಮತೆಯಿಗಾಗಿ
- Node.js/NPX - ಬಾಹ್ಯ MCP ಸರ್ವರ್ ಅನ್ನು ಚಲಾಯಿಸಲು

## ನೀವು ಕಲಿತದ್ದೇನು:
- MCP ಒಗ್ಗೂಡಿಕೆ: ಸೆಮಾಂಟಿಕ್ ಕರ್ಣಲ್ ಏಜೆಂಟ್‌ಗಳನ್ನು ಬಾಹ್ಯ MCP ಸರ್ವರ್‌ಗಳಿಗೆ ಸಂಪರ್ಕಿಸುವುದು
- ನೈಜ-ಸಮಯದ ಡೇಟಾ ಪ್ರವೇಶ: ನೈಜ Airbnb ಆಸ್ತಿಗಳನ್ನು ಲೈವ್ APIಗಳ ಮೂಲಕ ಹುಡುಕುವುದು
- ಪ್ರೋಟೋಕಾಲ್ ಸಂವಹನ: ಏಜೆಂಟ್ ಮತ್ತು MCP ಸರ್ವರ್ ನಡುವೆ stdio ಸಂವಹನ ಬಳಸುವುದು
- ಕಾರ್ಯ ಪತ್ತೆ: MCP ಸರ್ವರ್‌ಗಳಿಂದ ಲಭ್ಯವಿರುವ ಕಾರ್ಯಗಳನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಪತ್ತೆಹಚ್ಚುವುದು
- ಸ್ಟ್ರೀಮಿಂಗ್ ಪ್ರತಿಕ್ರಿಯೆಗಳು: ಕಾರ್ಯ ಕರೆಗಳನ್ನು ನೈಜ-ಸಮಯದಲ್ಲಿ ಸೆರೆಹಿಡಿದು ಲಾಗ್ ಮಾಡುವುದು
- HTML ರೆಂಡರಿಂಗ್: ಏಜೆಂಟ್ ಪ್ರತಿಕ್ರಿಯೆಗಳನ್ನು ಶೈಲೀಬದ್ಧ ಟೇಬಲ್‌ಗಳು ಮತ್ತು ಪರಸ್ಪರ ಪ್ರದರ್ಶನಗಳೊಂದಿಗೆ ಸ್ವರೂಪಗೊಳಿಸುವುದು

## ಮುಂದಿನ ಹಂತಗಳು:
- ಹೆಚ್ಚುವರಿ MCP ಸರ್ವರ್‌ಗಳನ್ನು (ಹವಾಮಾನ, ವಿಮಾನಗಳು, ರೆಸ್ಟೋರೆಂಟ್‌ಗಳು) ಒಗ್ಗೂಡಿಸಿ
- MCP ಮತ್ತು A2A ಪ್ರೋಟೋಕಾಲ್‌ಗಳನ್ನು ಸಂಯೋಜಿಸುವ ಬಹು-ಏಜೆಂಟ್ ವ್ಯವಸ್ಥೆಯನ್ನು ನಿರ್ಮಿಸಿ
- ನಿಮ್ಮದೇ ಡೇಟಾ ಮೂಲಗಳಿಗೆ ಕಸ್ಟಮ್ MCP ಸರ್ವರ್‌ಗಳನ್ನು ರಚಿಸಿ
- ಸೆಷನ್‌ಗಳಾದ್ಯಂತ ಸ್ಥಿರ ಸಂಭಾಷಣಾ ಮೆಮೊರಿಯನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸಿ
- MCP ಸರ್ವರ್ ಸಂಯೋಜನೆಯೊಂದಿಗೆ ಏಜೆಂಟ್ ಅನ್ನು Azure Functions ಗೆ ನಿಯೋಜಿಸಿ
- ಬಳಕೆದಾರ ಪ್ರಾಮಾಣೀಕರಣ ಮತ್ತು ಬುಕ್ಕಿಂಗ್ ಸಾಮರ್ಥ್ಯಗಳನ್ನು ಸೇರಿಸಿ


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸಮೀಕ್ಷೆ**:  
ಈ ದಸ್ತಾವೇಜನ್ನು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯನ್ನು ಸಾಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಮರ್ಪಕತೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಪ್ರಾಮಾಣಿಕ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪುಅರ್ಥಗಳು ಅಥವಾ ತಪ್ಪುಅರ್ಥೈಸುವಿಕೆಗೆ ನಾವು ಹೊಣೆಗಾರರಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
